In [ ]:
# !pip install --upgrade google-genai
# gcloud auth application-default login

In [ ]:
from google import genai
from google.genai import types
import base64

def generate():
  client = genai.Client(
      vertexai=True,
      project="testfab-362608",
      location="us-central1"
  )

  video1 = types.Part.from_uri(
      file_uri="gs://fab_public_bucket/video/fareed/Fareed_1_TR_AR.mp4",
      mime_type="video/mp4",
  )
  text1 = types.Part.from_text("""<INSTRUCTIONS> 
Analyze the input video to identify all the key moments, themes, and emotional arcs. 
Recommend a list of 4 top key relevant video frame clips timestamps from the given input video thoughout the complete video. 
The maximum duration (end_timestamp - start_timestamp) of video frame clip will always be below 15 seconds.
The last recommendation should be at the very end of the video.
</INSTRUCTIONS> 

<OUTPUT Format> 
JSON
[
{
start_timestamp: \\\"mm:ss\\\",
end_timestamp: \\\"mm:ss\\\",
reasoning: \\\"\\\"
}
]
</OUTPUT Format>""")

  model = "gemini-2.0-flash-exp"
  contents = [
    types.Content(
      role="user",
      parts=[
        video1,
        text1
      ]
    )
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    response_mime_type = "application/json"
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(type(chunk))

generate()